In [1]:
import sys
sys.path.append('../src/')
import data_io
import pandas as pd
import numpy as np
import parsing_utils as utils
import re

In [2]:
import imp
imp.reload(utils)

<module 'parsing_utils' from '/Users/Ryan/cancer-crowdfunding-explorer/notebooks/parsing_utils.py'>

## 1. Merge the output tables into a giant csv 

Optional: use tqdm to have progress bar of things that take long time by setting use_tqdm=True

In [3]:
use_tqdm=True
if use_tqdm: from tqdm.notebook import tqdm

### A. Only need to generate this once, can skip to part B if already have all_output.csv

In [ ]:
fiter=(data_io.input_raw/'scrape_output'/'done').glob('*.csv')
if use_tqdm: fiter = tqdm([*fiter])
dflist = []
for fp in fiter:
    if use_tqdm: fiter.set_description(f"Reading {fp.stem}")
    df = pd.read_csv(fp,encoding='utf-8',dtype=str)
    dflist.append(df.assign(i_filename=fp.stem))

dfs = pd.concat(dflist,ignore_index=True)
dfs.fillna('none',inplace=True)
# Save output
dfs.to_csv(data_io.input_raw/'scrape_output'/'all_output.csv',index=False,sep="|")

### B. If ran part A at some point before, only need to read in all_output.csv

In [ ]:
dfs = pd.read_csv(data_io.input_raw/'scrape_output'/'all_output.csv',encoding='utf-8',sep="|")

## 2. Prepare for removing duplicates

Create campaign_id column for each row

In [ ]:
# Clean gfm_url if value is not none, else left as none
if use_tqdm:
    tqdm.pandas(desc='Clean GFM url')
    dfs.gfm_url = dfs.gfm_url.where(dfs.gfm_url=='none',other=dfs.gfm_url.progress_apply(utils.clean_gfm_url))
    tqdm.pandas(desc='Get campaign_id')
    dfs = dfs.assign(campaign_id=dfs.gfm_url.progress_apply(utils.get_campaign_id))
else:
    dfs.gfm_url = dfs.gfm_url.where(dfs.gfm_url=='none',other=dfs.gfm_url.apply(utils.clean_gfm_url))
    dfs = dfs.assign(campaign_id=dfs.gfm_url.apply(utils.get_campaign_id))

In [ ]:
cid_counts = dfs.campaign_id.value_counts()
cid_counts.head(20)

## 3. Remove duplicates by campaign_id

Also set up an "exclusion df" to track the number of campaigns excluded

In [ ]:
exclusion_df = pd.DataFrame(columns=['original_campaign_count', 'duplicate_url', 'poor_wayback_qual',
                                    'duplicate_title_organizer_date_loc', 'state_not_us', 'not_USD',
                                    'not_cancer', 'tag_not_medical', 'year_is_null', 'failed_geocode',
                                    'last_null_county_check'], index=['deleted', 'total'])

In [ ]:
dfs_updated=dfs
before_drop= dfs_updated.shape
exclusion_df.loc['total', 'original_campaign_count'] = len(dfs)
dfs_unique_gfmurl = utils.keep_best_duplicate(dfs_updated,subset=['campaign_id'],use_tqdm=use_tqdm)
exclusion_df.loc['total', 'duplicate_url'] = len(dfs_unique_gfmurl)
exclusion_df.loc['deleted', 'duplicate_url'] = len(dfs) - len(dfs_unique_gfmurl)
after_drop = dfs_unique_gfmurl.shape
print(before_drop,after_drop)

Confirm that we kept the right duplicate 

In [ ]:
campaign_id = 'meadowhillfarmrescuefund' # one of the example 
dfs_unique_gfmurl[dfs_unique_gfmurl.campaign_id==campaign_id]
# should return the more recent duplicate (wayback_status == 'present:success' one)

From here on, we'll index the rows by their campaign_id since these should be unique now

In [ ]:
dfs_unique_gfmurl.set_index('campaign_id',inplace=True)
dfs_unique_gfmurl.to_csv(data_io.input_raw/'scrape_output'/'all_output_unique_gfm_url.csv',encoding='utf-8',sep='|')

## 4. Remove rows that failed completely during scraping 

In [7]:
dfs_unique_gfmurl = pd.read_csv(
    data_io.input_raw/'scrape_output'/'all_output_unique_gfm_url.csv',
    encoding='utf-8',sep='|',index_col=[0])

/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
dfs_unique_gfmurl.head()

,url,last_donation_time,last_update_time,created_date,location_city,location_country,location_postalcode,location_stateprefix,poster,description,...,num_donors,num_likes,num_shares,charity_details,error_message,archive_timestamp,query_url,gfm_url,wayback_status,i_filename
campaign_id,,,,,,,,,,,,,,,,,,,,,
help-our-knot-staff,http://web.archive.org/web/20200528174733/http...,20 hrs,none,"Created March 17, 2020","Evanston, IL",US,60201,none,none,none,...,186,none,none,none,none,20200528174733,http://web.archive.org/web/20200528174733/http...,https://www.gofundme.com/f/help-our-knot-staff,present: scraped but did not meet success crit...,master_scraped_output_i620000
support-bookhouse-servers-and-kitchen-staff,http://web.archive.org/web/20200618090038/http...,há um mês,none,Criada em 17 de Março de 2020,"Atlanta, GA",US,30306,none,none,none,...,58,none,none,none,none,20200618090038,http://web.archive.org/web/20200618090038/http...,https://www.gofundme.com/f/support-bookhouse-s...,present: scraped but did not meet success crit...,master_scraped_output_i620000
together-we-can-help-rj-beat-rare-sarcoma-cancer,https://www.gofundme.com/together-we-can-help-...,none,none,none,none,none,none,none,none,none,...,none,none,none,none,none,20210304172827,http://www.gofundme.com/together-we-can-help-r...,https://www.gofundme.com/together-we-can-help-...,present: scraped but did not meet success crit...,master_scraped_output_i620000
in-honor-of-neptune,https://www.gofundme.com/in-honor-of-neptune,none,none,none,none,none,none,none,none,none,...,none,none,none,none,none,20210304172828,http://www.gofundme.com/in-honor-of-neptune,https://www.gofundme.com/in-honor-of-neptune,present: scraped but did not meet success crit...,master_scraped_output_i620000
steve-needs-2-surgeries,http://web.archive.org/web/20200506210119/http...,6 d,none,"Created March 17, 2020","Rising Fawn, GA",US,30738,none,none,none,...,35,none,none,none,none,20200506210119,http://web.archive.org/web/20200506210119/http...,https://www.gofundme.com/f/steve-needs-2-surge...,present: scraped but did not meet success crit...,master_scraped_output_i620000


Get a sample of 5 campaigns for each type of wayback_status

In [9]:
dfs_unique_gfmurl['wayback_status_type'] = dfs_unique_gfmurl.wayback_status.apply(lambda x: ''.join([c for c in x if not c.isnumeric() ]))
wayback_types = dfs_unique_gfmurl.groupby('wayback_status_type').head(5)

Check for rows with request failed error, these urls should be rescraped, for now remove them from the master df

In [10]:
failed_request_error_bool = dfs_unique_gfmurl.title=='ERROR: The request could not be satisfied'
cols_to_check = ['last_update_time','created_date','location_city','location_country','poster','story','raised_amnt',
       'goal_amnt','goal','currency','num_donors','num_likes','num_shares','charity_details','error_message']
failed_request_error_bool2 = (dfs_unique_gfmurl[cols_to_check]=='none').all(axis=1)

# the really good scrape used window.initialState info and have standardized timestamps already for created_date
# we should scrape again to try to get the really good scrapes
created_date_ts = pd.to_datetime(dfs_unique_gfmurl.created_date, errors='coerce')
bad_scrape_bool = created_date_ts.isna()

to_scrape_again = dfs_unique_gfmurl[failed_request_error_bool | failed_request_error_bool2 | bad_scrape_bool]

to_scrape_again.to_csv(data_io.input_raw/'scrape_output'/'all_output_to_scrape_again.csv',encoding='utf-8',sep='|')
print(to_scrape_again.shape)

(212508, 28)


In [11]:
dfs_unique_gfmurl = dfs_unique_gfmurl[~(failed_request_error_bool | failed_request_error_bool2)]

Types of wayback status there are:

In [12]:
wayback_status_type_counts=wayback_types.wayback_status_type.value_counts().sort_index(ascending=False)
print(wayback_status_type_counts)
all_wayback_status_types=wayback_status_type_counts.index.values

present: success                                                                                           5
present: scraped but did not meet success criteria ; wayback: url not found in archives                    5
present: scraped but did not meet success criteria ; wayback: success                                      5
present: scraped but did not meet success criteria ; wayback: scraped but did not meet success standard    5
present: scraped but did not meet success criteria ; wayback: request failed                               5
present: scraped but did not meet success criteria ; wayback: inactive                                     5
present: request failed ; wayback: url not found in archives                                               5
present: request failed ; wayback: success                                                                 5
present: inactive ; wayback: url not found in archives                                                     5
present: inactive ;

To view example of a type

In [13]:
i=1
print(all_wayback_status_types[i])
_m=wayback_types.wayback_status_type==all_wayback_status_types[i]
wayback_types[_m].iloc[2]

present: scraped but did not meet success criteria ; wayback: url not found in archives


url                       https://www.gofundme.com/support-tabs-bar-staff
last_donation_time                                                   none
last_update_time                                                     none
created_date                                                         none
location_city                                                        none
location_country                                                     none
location_postalcode                                                  none
location_stateprefix                                                 none
poster                                                               none
description                                                          none
story                                                                none
title                           ERROR: The request could not be satisfied
goal                                                                 none
raised_amnt                           

Construct a list of wayback_status_type that can be consider decently successful,
we don't consider `wayback: scraped but did not meet success standard` as successful because usually these are redirects to gfm homepage not actual campaign webpage

In [14]:
allowed_wayback_status_types = ['present: success',
                                'present: scraped but did not meet success criteria ; wayback: success',
                                'present: scraped but did not meet success criteria ; wayback: inactive',
                                'present: request failed ; wayback: success',
                                'present: request failed ; wayback: inactive'
                               ]
allowed_wayback_status_types += ['present: inactive ; wayback: url not found in archives',
                                'present: inactive ; wayback: success',
                                'present: inactive ; wayback: scraped but did not meet success standard',
                                'present: inactive ; wayback: request failed',
                                'present: inactive ; wayback: no working archives out of  archives',
                                'present: inactive ; wayback: inactive']
allowed_wayback_status_types += ['present: campaign not found ; wayback: success',
                                'present: campaign not found ; wayback: inactive']
allowed_wayback_status_types += ['none'] 

In [15]:
dfs_all_decent = dfs_unique_gfmurl.loc[dfs_unique_gfmurl.wayback_status_type.isin(allowed_wayback_status_types),:]
#exclusion_df.loc['total', 'poor_wayback_qual'] = len(dfs_all_decent)
#exclusion_df.loc['deleted', 'poor_wayback_qual'] = len(dfs_unique_gfmurl) - len(dfs_all_decent)
print(dfs_unique_gfmurl.shape,dfs_all_decent.shape)

NameError: name 'exclusion_df' is not defined

Distribution of wayback status type

In [16]:
dfs_all_decent.wayback_status_type.value_counts()

present: success                                                          1483941
present: inactive ; wayback: url not found in archives                      73283
present: scraped but did not meet success criteria ; wayback: success       31841
present: inactive ; wayback: success                                        26543
present: inactive ; wayback: inactive                                        4199
present: scraped but did not meet success criteria ; wayback: inactive        588
present: inactive ; wayback: request failed                                   435
present: campaign not found ; wayback: success                                390
present: request failed ; wayback: success                                     68
present: inactive ; wayback: scraped but did not meet success standard         41
present: campaign not found ; wayback: inactive                                10
none                                                                            1
Name: wayback_st

Check the type of wayback_status_type that were excluded, make sure there's no type we still want to keep 

In [17]:
wayback_types.wayback_status_type[~wayback_types.wayback_status_type.isin(dfs_all_decent.wayback_status_type.unique())].unique()

array(['present: scraped but did not meet success criteria ; wayback: url not found in archives',
       'present: scraped but did not meet success criteria ; wayback: scraped but did not meet success standard',
       'present: campaign not found ; wayback: url not found in archives',
       'present: scraped but did not meet success criteria ; wayback: request failed',
       'present: request failed ; wayback: url not found in archives',
       'present: campaign not found ; wayback: request failed',
       'present: inactive ; wayback: campaign not found',
       'present: campaign not found ; wayback: campaign not found',
       'present: campaign not found ; wayback: scraped but did not meet success standard'],
      dtype=object)

In [18]:
dfs_all_decent.drop(columns=['wayback_status_type'],inplace=True)
dfs_all_decent.to_csv(data_io.input_raw/'scrape_output'/'all_output_successful.csv',
                      encoding='utf-8',sep='|')

/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 5. Clean and parse columns 

In [19]:
%%time
dfs_all_decent = pd.read_csv(data_io.input_raw/'scrape_output'/'all_output_successful.csv',
                             index_col=[0],encoding='utf-8',sep='|')

<string>:2: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 32.8 s, sys: 17 s, total: 49.8 s
Wall time: 2min 54s


In [20]:
dfs_all_decent.columns

Index(['url', 'last_donation_time', 'last_update_time', 'created_date',
       'location_city', 'location_country', 'location_postalcode',
       'location_stateprefix', 'poster', 'description', 'story', 'title',
       'goal', 'raised_amnt', 'goal_amnt', 'currency', 'tag', 'num_donors',
       'num_likes', 'num_shares', 'charity_details', 'error_message',
       'archive_timestamp', 'query_url', 'gfm_url', 'wayback_status',
       'i_filename'],
      dtype='object')

In [22]:
dfs_all_decent.head()

,url,last_donation_time,last_update_time,created_date,location_city,location_country,location_postalcode,location_stateprefix,poster,description,...,num_donors,num_likes,num_shares,charity_details,error_message,archive_timestamp,query_url,gfm_url,wayback_status,i_filename
campaign_id,,,,,,,,,,,,,,,,,,,,,
help-our-knot-staff,http://web.archive.org/web/20200528174733/http...,20 hrs,none,"Created March 17, 2020","Evanston, IL",US,60201,none,none,none,...,186,none,none,none,none,20200528174733,http://web.archive.org/web/20200528174733/http...,https://www.gofundme.com/f/help-our-knot-staff,present: scraped but did not meet success crit...,master_scraped_output_i620000
support-bookhouse-servers-and-kitchen-staff,http://web.archive.org/web/20200618090038/http...,há um mês,none,Criada em 17 de Março de 2020,"Atlanta, GA",US,30306,none,none,none,...,58,none,none,none,none,20200618090038,http://web.archive.org/web/20200618090038/http...,https://www.gofundme.com/f/support-bookhouse-s...,present: scraped but did not meet success crit...,master_scraped_output_i620000
steve-needs-2-surgeries,http://web.archive.org/web/20200506210119/http...,6 d,none,"Created March 17, 2020","Rising Fawn, GA",US,30738,none,none,none,...,35,none,none,none,none,20200506210119,http://web.archive.org/web/20200506210119/http...,https://www.gofundme.com/f/steve-needs-2-surge...,present: scraped but did not meet success crit...,master_scraped_output_i620000
mac039s-wood-grilled-staff-relief-fund,http://web.archive.org/web/20201223192038/http...,3 mos,2020-09-30T18:57:33-05:00,2020-03-17T21:05:17-05:00,"Chicago, IL",US,none,IL,Macs Wood Grilled,none,...,65,none,none,none,none,20201223192038,http://web.archive.org/web/20201223192038/http...,https://www.gofundme.com/f/mac039s-wood-grille...,present: scraped but did not meet success crit...,master_scraped_output_i620000
people-helpers-food-for-school-kids,http://web.archive.org/web/20201021042735/http...,5 mos,2020-03-23T18:49:51-05:00,2020-03-17T21:25:19-05:00,"Saint Charles, MO",US,none,MO,Tim Randazzo,none,...,10,none,none,"{'id': 374326, 'npo_id': '1232740', 'ein': '46...",none,20201021042735,http://web.archive.org/web/20201021042735/http...,https://www.gofundme.com/f/people-helpers-food...,present: scraped but did not meet success crit...,master_scraped_output_i620000


### Seperate out the really good scrapes that don't need parsing of title, date, and goal amount

In [23]:
# the really good scrape used window.initialState info and have standardized timestamps already for created_date
created_date_ts = pd.to_datetime(dfs_all_decent.created_date, errors='coerce')
dfs_all_decent_good_scrape_bool = created_date_ts.notna()

# save these seperately cause we don't have to process them 
dfs_good_scrape = dfs_all_decent[dfs_all_decent_good_scrape_bool]
dfs_all_decent =  dfs_all_decent[~dfs_all_decent_good_scrape_bool]
# dfs_all_decent are the ones we need to parse out information from
print(dfs_good_scrape.shape,dfs_all_decent.shape)

(1540983, 27) (80357, 27)


### Parse out title for benefiter, organizer, and campaign_title

In [24]:
if use_tqdm: 
    tqdm.pandas(desc='Parsing title')
    title_parsed_dicts=dfs_all_decent.title.progress_apply(utils.parse_title)
else:
    title_parsed_dicts=dfs_all_decent.title.apply(utils.parse_title)
title_parsed_df=pd.DataFrame.from_records(title_parsed_dicts,index=title_parsed_dicts.index).dropna(how='all',axis=0)
title_parsed_df.head()

Parsing title:   0%|          | 0/80357 [00:00<?, ?it/s]

,benefiter,organizer,campaign_title,campaign_title_type
campaign_id,,,,
77okis,NaN,Kahlil Almustafa,Family First Foreclosure Defense,campaign
77pkok,NaN,Debbie Lacina Dramstad,support patton douglas leukemia battle,campaign
77tp2c,NaN,Rebecca Bandy,Becca's Trip to Cambodia with PCL,campaign
77x39g,NaN,Sean Brown,Support Don Mayo's Family,campaign
support-my-student-jesus,NaN,Dawn Addis,My student Jesus is ill - please support,campaign


Join parsed results into df

In [25]:
parsed_cols = ['poster','title']
# change column name organizer -> poster and keeping everything else the same
title_parsed_df.columns = title_parsed_df.columns.map(
    title_parsed_df.columns.to_series().replace({'organizer':'poster','campaign_title':'title'}).to_dict())

dfs_all_decent.drop(columns=parsed_cols,errors='ignore',inplace=True)
dfs_all_decent = dfs_all_decent.merge(title_parsed_df[parsed_cols],right_index=True,left_index=True,how='left',indicator=True)
# See merge results
print(dfs_all_decent._merge.value_counts())
dfs_all_decent.drop(columns='_merge',inplace=True)

left_only     53285
both          27072
right_only        0
Name: _merge, dtype: int64


### Parse day, month & year from created_date

In [26]:
if use_tqdm: 
    tqdm.pandas(desc='Parsing date')
    date_parsed_dicts = dfs_all_decent.created_date.progress_apply(utils.parse_created_date)
else:
    date_parsed_dicts = dfs_all_decent.created_date.apply(utils.parse_created_date)
date_parsed_df = pd.DataFrame.from_records(date_parsed_dicts,index=date_parsed_dicts.index)   

Parsing date:   0%|          | 0/80357 [00:00<?, ?it/s]

failed to parse Created 4 days ago
failed to parse Created 4 days ago
failed to parse Created 5 days ago
failed to parse Created 3 days ago
failed to parse Created 7 days ago
failed to parse Created 7 days ago
failed to parse Created 6 days ago
failed to parse Created 6 days ago
failed to parse Created 1 day ago
failed to parse Created 2 days ago
failed to parse Created 1 day ago
failed to parse Created 4 days ago
failed to parse Created 4 days ago
failed to parse Created 6 days ago
failed to parse Created 4 days ago
failed to parse Created 7 days ago
failed to parse Created 7 days ago
failed to parse Created 4 days ago
failed to parse Created 7 days ago
failed to parse Created 3 days ago
failed to parse Created 3 days ago
failed to parse Created 7 days ago
failed to parse Created 2 days ago
failed to parse Created 6 days ago
failed to parse Created 6 days ago
failed to parse Created 5 days ago
failed to parse Created 6 days ago
failed to parse Created 17 hours ago
failed to parse Crea

failed to parse Created 5 days ago
failed to parse Created 7 hours ago
failed to parse Created 2 days ago
failed to parse Created 3 days ago
failed to parse bedding
failed to parse Created 6 days ago
failed to parse Création il y a 2 jours
failed to parse Created 2 days ago
failed to parse Created 3 days ago
failed to parse Created 6 days ago
failed to parse Created 1 day ago
failed to parse Just now
failed to parse Just now
failed to parse Created 5 days ago
failed to parse Created 7 days ago
failed to parse Created 4 days ago
failed to parse Created 7 days ago
failed to parse Created 21 hours ago
failed to parse Created 2 days ago
failed to parse Created 3 days ago
failed to parse Created 2 days ago
failed to parse Created 2 days ago
failed to parse Created 2 days ago
failed to parse Created 6 days ago
failed to parse Created 2 days ago
failed to parse Created 2 days ago
failed to parse Created 4 days ago
failed to parse Created 3 days ago
failed to parse Created 1 day ago
failed to

failed to parse Created 7 days ago
failed to parse Created 1 hour ago
failed to parse Created 3 hours ago
failed to parse Created 2 days ago
failed to parse Created 13 hours ago
failed to parse Creata 7 giorni fa
failed to parse Created 6 days ago
failed to parse Created 4 days ago
failed to parse Created 5 days ago
failed to parse Created 6 days ago
failed to parse Created 3 days ago
failed to parse Created 6 days ago
failed to parse Created 5 hours ago
failed to parse Created 7 days ago
failed to parse 5 dagen geleden gemaakt
failed to parse Création il y a 4 jours
failed to parse Created 1 day ago
failed to parse Created 3 days ago
failed to parse Created 2 days ago
failed to parse Created 19 hours ago
failed to parse Created 6 days ago
failed to parse Created 7 days ago
failed to parse Created 4 days ago
failed to parse Created 6 days ago
failed to parse Created 7 days ago
failed to parse Created 4 days ago
failed to parse Created 2 days ago
failed to parse Created 5 days ago
fail

Fix the "days ago" and "hours ago" issue in some rows

In [27]:
ago_re_pattern = r'Created +(\d+ +(days?|hours?)) +ago'
ago_m = dfs_all_decent.created_date.apply(lambda x: re.search(ago_re_pattern,x) is not None)

ago_rows = dfs_all_decent.loc[ago_m,'created_date']
print(ago_rows.head())

time_delta_ago = ago_rows.apply(lambda x: re.findall(ago_re_pattern,x)[0][0])
time_delta_ago = pd.to_timedelta(time_delta_ago)
print(time_delta_ago.head())

archived_ts = pd.to_datetime(dfs_all_decent.loc[ago_rows.index, 'archive_timestamp'].astype(str))
created_date_ts = archived_ts - time_delta_ago

parsed_ago_rows = pd.DataFrame({'day':created_date_ts.dt.day,
    'month':created_date_ts.dt.month_name(),
    'year':created_date_ts.dt.year
    },index=created_date_ts.index)
print(parsed_ago_rows.head()) 

# Update day month year fields 
print(date_parsed_df[ago_m].head())
date_parsed_df.update(parsed_ago_rows,overwrite=True)
print(date_parsed_df[ago_m].head())


campaign_id
silverback-distillery-sanitizer-supplies      Created 4 days ago
make-hand-sanitizer-for-pittsburgh            Created 4 days ago
restaurant-st-tropez-employees-fund           Created 5 days ago
doctors-without-masksmedecins-sans-masques    Created 3 days ago
custodians-of-shelburne-community-school      Created 7 days ago
Name: created_date, dtype: object
campaign_id
silverback-distillery-sanitizer-supplies     4 days
make-hand-sanitizer-for-pittsburgh           4 days
restaurant-st-tropez-employees-fund          5 days
doctors-without-masksmedecins-sans-masques   3 days
custodians-of-shelburne-community-school     7 days
Name: created_date, dtype: timedelta64[ns]
                                            day  month  year
campaign_id                                                 
silverback-distillery-sanitizer-supplies     19  March  2020
make-hand-sanitizer-for-pittsburgh           20  March  2020
restaurant-st-tropez-employees-fund          20  March  2020
doctors-w

Map non-English months to standard English months

In [28]:
month_map_dict = {'février':'february','octobre':'october','juli':'july','junho':'june','09':'september','abril':'april',
'março':'march','mars':'march','februar':'february','januar':'january', 'avril':'april','juin':'june',
'juillet':'july','augustus':'august','mai':'may','mai':'may','märz':'march','juni':'June',
'settembre':'september','gennaio':'january','septiembre':'september','mayo':'may',
'décembre':'december','nisan':'April','maggio':'may','febbraio':'february',
'marzo':'march','janvier':'january','dezember':'december','novembro':'november',
'febrero':'february','aprile':'april','maio':'may','novembre':'november','mei':'may',
'septembre':'september','oktober':'october','junio':'june','enero':'january','februari':'february','januari':'january',
'fevereiro':'february','noviembre':'november','giugno':'june','agosto':'august'}

In [29]:
# map values
mapped = date_parsed_df.month.str.lower().map(month_map_dict).str.capitalize()
# if value was mapped keep the value, else keep original 
print(date_parsed_df[mapped.notna()].head())
date_parsed_df.month=mapped.where(mapped.notna(),date_parsed_df.month).str.capitalize()
print(date_parsed_df[mapped.notna()].head())

                                       day  month  year
campaign_id                                            
north-texas-support-for-coffee          18   mars  2020
fort-wayne-bar-aid                      18  marzo  2020
bird-rock-coffee-baristas-bressi-ranch  18  marzo  2020
coaches-vs-covid19                      19  marzo  2020
maui-face-mask-manufacture              21   mars  2020
                                       day  month  year
campaign_id                                            
north-texas-support-for-coffee          18  March  2020
fort-wayne-bar-aid                      18  March  2020
bird-rock-coffee-baristas-bressi-ranch  18  March  2020
coaches-vs-covid19                      19  March  2020
maui-face-mask-manufacture              21  March  2020


Join parsed results into df

In [30]:
dfs_all_decent = dfs_all_decent.merge(date_parsed_df[['day','month','year']],right_index=True,left_index=True,how='left',indicator=True)
# See merge results
print(dfs_all_decent._merge.value_counts())
dfs_all_decent.drop(columns='_merge',inplace=True)
dfs_all_decent.loc[:,['created_date','day','month','year']].head()

both          80357
left_only         0
right_only        0
Name: _merge, dtype: int64


,created_date,day,month,year
campaign_id,,,,
help-our-knot-staff,"Created March 17, 2020",17,March,2020
support-bookhouse-servers-and-kitchen-staff,Criada em 17 de Março de 2020,17,Março,2020
steve-needs-2-surgeries,"Created March 17, 2020",17,March,2020
draught-house-pub-employee-fund,"Created March 17, 2020",17,March,2020
douglas-county-cares,"Created March 17, 2020",17,March,2020


### Parse goal into raised and goal amount 

Parse string inside 'raised' and 'goal' into amount and currency

Note: will fail for values where money was mentioned but not specified if it was raised or goal (e.g. '$8,130') 

In [31]:
if use_tqdm:
    tqdm.pandas(desc='Parsing goal')
    goal_parsed_dicts = dfs_all_decent.goal.progress_apply(utils.get_raised_and_goal_amount,**dict(USD_only=False))
else:
    goal_parsed_dicts = dfs_all_decent.goal.apply(utils.get_raised_and_goal_amount,**dict(USD_only=False))
# toogle USD_only to parse euro,pound or % also, else it'd return np.nan for these fields
goal_parsed_df = pd.DataFrame.from_records(goal_parsed_dicts.to_list(),index=goal_parsed_dicts.index)
goal_parsed_df.head()

Parsing goal:   0%|          | 0/80357 [00:00<?, ?it/s]

failed to parse 26


,raised,goal,raised_amnt,raised_curr,goal_amnt,goal_curr
campaign_id,,,,,,
help-our-knot-staff,NaN,NaN,NaN,NaN,NaN,NaN
support-bookhouse-servers-and-kitchen-staff,NaN,NaN,NaN,NaN,NaN,NaN
steve-needs-2-surgeries,NaN,NaN,NaN,NaN,NaN,NaN
draught-house-pub-employee-fund,NaN,NaN,NaN,NaN,NaN,NaN
douglas-county-cares,NaN,NaN,NaN,NaN,NaN,NaN


Check to see locations of campaigns that are not USD to see if they're based in the US

In [32]:
not_usd_m = (goal_parsed_df.raised_curr.str.find('$')==-1) & (goal_parsed_df.goal_curr.str.find('$')==-1) 
#not_usd_m = not_usd_m & goal_parsed_df.raised_curr.notna() & goal_parsed_df.goal_curr.notna()
# look at location of these campaigns
not_usd_locs=dfs_all_decent.loc[not_usd_m,'location_city'].str.lower().value_counts()

From manual checking, at least the first 180 locations are not but there are almost 20,000 locations.

In [33]:
not_usd_locs.head(20)

london, greater london, united kingdom            65
dublin, ireland                                   37
ireland                                           35
madrid, m, spain                                  18
berlin, deutschland                               16
paris, france                                     12
none                                              11
barcelona, ct, spain                               8
glasgow, scotland, united kingdom                  7
brighton, south east england, united kingdom       6
manchester, north west england, united kingdom     6
münchen, bayern, deutschland                      5
birmingham, west midlands, united kingdom          4
cork city, cork ireland                            4
liverpool, north west england, united kingdom      4
hamburg, deutschland                               4
frankfurt am main, hessen, deutschland             3
galway city, galway ireland                        3
london, eng                                   

Do a code check using get_state_var() & state_is_valid()

In [34]:
valid_US_state_m = not_usd_locs.index.to_series().apply(utils.get_state_var).apply(utils.state_is_valid)
# show any potential US location that have non_USD values
not_usd_locs[valid_US_state_m]

Series([], Name: location_city, dtype: int64)

Check on values such as '2.2k','3M','1.5B' to make sure they're parsed correctly

In [35]:
pd.concat([goal_parsed_df.loc[goal_parsed_df.goal.str.find('k')>-1,:].head(3),
goal_parsed_df.loc[goal_parsed_df.goal.str.lower().str.find('m')>-1,:].head(3),
goal_parsed_df.loc[goal_parsed_df.goal.str.lower().str.find('b')>-1,:].head(3)])

,raised,goal,raised_amnt,raised_curr,goal_amnt,goal_curr
campaign_id,,,,,,
78gbm0,"$2,575",$15k,2575.000,$,1.500000e+04,$
78kap0,€37.774,€50k,37.774,€,5.000000e+04,€
792vz4,"$10,628",$25k,10628.000,$,2.500000e+04,$
legacycalpilots,"$111,022",$1.0M,111022.000,$,1.000000e+06,$
8b7go0,"$1,260",1M,1260.000,$,1.000000e+06,
mr-virgil-poole-we-love-you,"$2,880",$1.0M,2880.000,$,1.000000e+06,$
support-for-glenn-beck,"$11,630",$1.0B,11630.000,$,1.000000e+09,$


For goal currency that is NaN, extract from raised currency

In [36]:
goal_parsed_df.goal_curr = goal_parsed_df.goal_curr.where(
    goal_parsed_df.goal_curr.str.strip().replace('',np.nan).notna(),goal_parsed_df.raised_curr)

Join parsed results into df

In [37]:
parsed_goal_cols = ['raised_amnt','goal_amnt','currency']

# change column name organizer -> poster and keeping everything else the same
goal_parsed_df.columns = goal_parsed_df.columns.map(
    goal_parsed_df.columns.to_series().replace({'goal_curr':'currency'}).to_dict())

dfs_all_decent.drop(columns=parsed_goal_cols,errors='ignore',inplace=True) # drop if exist
dfs_all_decent = dfs_all_decent.merge(goal_parsed_df[parsed_goal_cols],right_index=True,left_index=True,how='left',indicator=True)
# See merge results
print(dfs_all_decent._merge.value_counts())
dfs_all_decent.drop(columns='_merge',inplace=True)
dfs_all_decent.loc[:,['goal','raised_amnt','goal_amnt','currency']].head()

both          80357
left_only         0
right_only        0
Name: _merge, dtype: int64


,goal,raised_amnt,goal_amnt,currency
campaign_id,,,,
help-our-knot-staff,none,NaN,NaN,NaN
support-bookhouse-servers-and-kitchen-staff,none,NaN,NaN,NaN
steve-needs-2-surgeries,none,NaN,NaN,NaN
draught-house-pub-employee-fund,none,NaN,NaN,NaN
douglas-county-cares,none,NaN,NaN,NaN


Make sure non-english goals got parsed correctly

In [38]:
goal_format_example = 'obiettivo'
goal_f_m=dfs_all_decent.goal.str.find(goal_format_example)>-1
dfs_all_decent.loc[goal_f_m,['goal','raised_amnt','goal_amnt','currency']]

,goal,raised_amnt,goal_amnt,currency
campaign_id,,,,


### Parse num_donors to get number of contributors

In [39]:
if use_tqdm:
    tqdm.pandas(desc='Standardizing status')
    s_status = dfs_all_decent.num_donors.progress_apply(utils.standardize_status)
    tqdm.pandas(desc='Parsing status')
    status_parsed_dicts = s_status.progress_apply(utils.parse_status)
else:
    s_status = dfs_all_decent.num_donors.apply(utils.standardize_status)
    status_parsed_dicts = s_status.apply(utils.parse_status)
status_parsed_df = pd.DataFrame.from_records(status_parsed_dicts.to_list(),index=status_parsed_dicts.index)
status_parsed_df['num_contributors'] = pd.to_numeric(status_parsed_df.ndonor.apply(utils.standardize_MBk_in_number_str))
status_parsed_df.head()

Standardizing status:   0%|          | 0/80357 [00:00<?, ?it/s]

Parsing status:   0%|          | 0/80357 [00:00<?, ?it/s]

failed to parse arrecadados por 30 pessoas em 1 mês


,ndonor,num_contributors
campaign_id,,
help-our-knot-staff,186,186.0
support-bookhouse-servers-and-kitchen-staff,58,58.0
steve-needs-2-surgeries,35,35.0
draught-house-pub-employee-fund,133,133.0
douglas-county-cares,16,16.0


Check on values such as '2.2k','3M','1.5B' to make sure they're parsed correctly

In [40]:
pd.concat([status_parsed_df.loc[status_parsed_df.ndonor.str.find('k')>-1,:].head(3),
status_parsed_df.loc[status_parsed_df.ndonor.str.lower().str.find('m')>-1,:].head(3),
status_parsed_df.loc[status_parsed_df.ndonor.str.lower().str.find('b')>-1,:].head(3)])

,ndonor,num_contributors
campaign_id,,


Join parsed results into df

In [41]:
parsed_status_cols = ['num_donors']

# change column name organizer -> poster and keeping everything else the same
status_parsed_df.columns = status_parsed_df.columns.map(
    status_parsed_df.columns.to_series().replace({'num_contributors':'num_donors'}).to_dict())

dfs_all_decent.drop(columns=parsed_status_cols,errors='ignore',inplace=True) # drop if exist
dfs_all_decent = dfs_all_decent.merge(status_parsed_df[parsed_status_cols],
              right_index=True,left_index=True,how='left',indicator=True)

# See merge results
print(dfs_all_decent._merge.value_counts())
dfs_all_decent.drop(columns='_merge',inplace=True)
dfs_all_decent.loc[:,['num_donors']].head()

both          80357
left_only         0
right_only        0
Name: _merge, dtype: int64


,num_donors
campaign_id,
help-our-knot-staff,186.0
support-bookhouse-servers-and-kitchen-staff,58.0
steve-needs-2-surgeries,35.0
draught-house-pub-employee-fund,133.0
douglas-county-cares,16.0


### Merge back with good scrapes

In [42]:
# merge with the good scrape dfs before merging
# create day,month,year for the good scrape df
# convert to pd.Timestamp, there's timezone info so has to convert to UTC then EST then remove tz info
created_date_ts = pd.to_datetime(dfs_good_scrape.created_date,utc=True,errors='coerce'
                                ).dt.tz_convert('EST').dt.tz_localize(None)
dfs_good_scrape['day'] = created_date_ts.dt.day
dfs_good_scrape['month'] = created_date_ts.dt.month_name()
dfs_good_scrape['year'] = created_date_ts.dt.year
print(dfs_good_scrape[['day','month','year']].head())

                                                    day  month  year
campaign_id                                                         
mac039s-wood-grilled-staff-relief-fund               17  March  2020
people-helpers-food-for-school-kids                  17  March  2020
qrmz7-a-friend-in-need                               19  March  2020
6c7yj-help-feed-the-homeless-during-covid19-qua...   17  March  2020
broadside-employees                                  20  March  2020


In [43]:
# merge together
print(dfs_good_scrape.shape,dfs_all_decent.shape)
test_concat = pd.concat([dfs_good_scrape,dfs_all_decent],axis=0,ignore_index=False)

(1540983, 30) (80357, 30)


In [44]:
dfs_all_decent = test_concat

In [45]:
dfs_all_decent.to_csv(
    data_io.input_raw/'scrape_output'/'dfs_all_decent_before_clean_location.csv',
                      encoding='utf-8',sep='|')

### Clean location columns

In [4]:
%%time
dfs_all_decent = pd.read_csv(
    data_io.input_raw/'scrape_output'/'dfs_all_decent_before_clean_location.csv',
                     index_col=[0], encoding='utf-8',sep='|',low_memory=False)

CPU times: user 45 s, sys: 1min 19s, total: 2min 4s
Wall time: 7min 24s


In [5]:
dfs_all_decent.columns

Index(['url', 'last_donation_time', 'last_update_time', 'created_date',
       'location_city', 'location_country', 'location_postalcode',
       'location_stateprefix', 'poster', 'description', 'story', 'title',
       'goal', 'raised_amnt', 'goal_amnt', 'currency', 'tag', 'num_donors',
       'num_likes', 'num_shares', 'charity_details', 'error_message',
       'archive_timestamp', 'query_url', 'gfm_url', 'wayback_status',
       'i_filename', 'day', 'month', 'year'],
      dtype='object')

In [6]:
dfs_all_decent.head()

,url,last_donation_time,last_update_time,created_date,location_city,location_country,location_postalcode,location_stateprefix,poster,description,...,charity_details,error_message,archive_timestamp,query_url,gfm_url,wayback_status,i_filename,day,month,year
campaign_id,,,,,,,,,,,,,,,,,,,,,
mac039s-wood-grilled-staff-relief-fund,http://web.archive.org/web/20201223192038/http...,3 mos,2020-09-30T18:57:33-05:00,2020-03-17T21:05:17-05:00,"Chicago, IL",US,none,IL,Macs Wood Grilled,none,...,none,none,20201223192038,http://web.archive.org/web/20201223192038/http...,https://www.gofundme.com/f/mac039s-wood-grille...,present: scraped but did not meet success crit...,master_scraped_output_i620000,17.0,March,2020.0
people-helpers-food-for-school-kids,http://web.archive.org/web/20201021042735/http...,5 mos,2020-03-23T18:49:51-05:00,2020-03-17T21:25:19-05:00,"Saint Charles, MO",US,none,MO,Tim Randazzo,none,...,"{'id': 374326, 'npo_id': '1232740', 'ein': '46...",none,20201021042735,http://web.archive.org/web/20201021042735/http...,https://www.gofundme.com/f/people-helpers-food...,present: scraped but did not meet success crit...,master_scraped_output_i620000,17.0,March,2020.0
qrmz7-a-friend-in-need,https://www.gofundme.com/f/qrmz7-a-friend-in-need,11 mos,2020-04-08T12:52:48-05:00,2020-03-19T16:52:36-05:00,"Gainesville, FL",US,none,FL,Joanna Caputa,none,...,none,none,20210304172924,http://www.gofundme.com/f/qrmz7-a-friend-in-need,https://www.gofundme.com/f/qrmz7-a-friend-in-need,present: success,master_scraped_output_i620000,19.0,March,2020.0
6c7yj-help-feed-the-homeless-during-covid19-quarantine,https://www.gofundme.com/f/6c7yj-help-feed-the...,10 mos,2020-09-25T12:06:50-05:00,2020-03-17T21:21:29-05:00,"Bridgewater, NJ",US,none,NJ,Tushan Dargan,none,...,"{'id': 283714, 'npo_id': '1441794', 'ein': '22...",none,20210304172933,http://www.gofundme.com/f/6c7yj-help-feed-the-...,https://www.gofundme.com/f/6c7yj-help-feed-the...,present: success,master_scraped_output_i620000,17.0,March,2020.0
broadside-employees,https://www.gofundme.com/f/broadside-employees,5 mos,2021-02-19T11:17:24-06:00,2020-03-20T19:55:35-05:00,"Boston, MA",US,none,MA,Kim Delaney,none,...,none,none,20210304172939,http://www.gofundme.com/f/broadside-employees,https://www.gofundme.com/f/broadside-employees,present: success,master_scraped_output_i620000,20.0,March,2020.0


Get only potential US campaigns

In [7]:
dfs_all_decent.location_country =  dfs_all_decent.location_country.replace('none',np.nan)
dfs_US = dfs_all_decent[(dfs_all_decent.location_country=='US') | (dfs_all_decent.location_country.isna())]
print(dfs_US.shape,dfs_all_decent.shape)

(1370628, 30) (1621340, 30)


In [8]:
dfs_US.location_state_prefix = dfs_US.location_stateprefix.replace('none',np.nan)

# parse in state info if its missing 
cleaned_location_city_info = dfs_US['location_city'].apply(utils.remove_non_loc_info)
parsed_state_info = cleaned_location_city_info.apply(utils.get_state_var)
dfs_US.location_stateprefix = dfs_US.location_stateprefix.where(
                                    dfs_US.location_state_prefix.notna(),parsed_state_info)
#parse city info
dfs_US['location_city_only'] = cleaned_location_city_info.apply(utils.get_other_loc).str.title()

dfs_US.head()[['location_city','location_city_only','location_stateprefix','location_country']]

/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,location_city,location_city_only,location_stateprefix,location_country
campaign_id,,,,
mac039s-wood-grilled-staff-relief-fund,"Chicago, IL",Chicago,IL,US
people-helpers-food-for-school-kids,"Saint Charles, MO",Saint Charles,MO,US
qrmz7-a-friend-in-need,"Gainesville, FL",Gainesville,FL,US
6c7yj-help-feed-the-homeless-during-covid19-quarantine,"Bridgewater, NJ",Bridgewater,NJ,US
broadside-employees,"Boston, MA",Boston,MA,US


In [9]:
dfs_US.to_csv(data_io.input_raw/'scrape_output'/'dfs_US.csv',encoding='utf-8',sep='|')

Update parsed info into our main dataframe

In [10]:
dfs_US = pd.read_csv(data_io.input_raw/'scrape_output'/'dfs_US.csv',
                     index_col=[0],encoding='utf-8',sep='|',low_memory=False)

In [11]:
%%time
dfs_all_decent.update(dfs_US,overwrite=True)

CPU times: user 6.6 s, sys: 24.3 s, total: 30.9 s
Wall time: 2min 45s


In [12]:
%%time
#### Update parsed info into our main dataframe
dfs_all_decent = dfs_all_decent.merge(
    dfs_US[['location_city_only']],left_index=True,right_index=True,how='left')
del dfs_US

CPU times: user 1.81 s, sys: 4.21 s, total: 6.02 s
Wall time: 23 s


In [13]:
%%time
### Clean social media info
dfs_all_decent['likes'] = dfs_all_decent['num_likes'].apply(utils.get_social)
dfs_all_decent['shares'] = dfs_all_decent['num_shares'].apply(utils.get_social)

thousand_bool = dfs_all_decent.num_shares.str.find('k')>-1
print(dfs_all_decent.loc[thousand_bool,['num_likes','num_shares','num_donors','likes','shares']].head(10))

dfs_all_decent.num_likes = dfs_all_decent.likes
dfs_all_decent.num_shares = dfs_all_decent.shares
dfs_all_decent.drop(columns=['likes','shares'],inplace=True)

                      num_likes num_shares  num_donors likes shares
campaign_id                                                        
7hy03o                     1.6k       7.6k      1677.0  1600   7600
7ixnhk                     1.1k       2.4k      1193.0  1100   2400
across-america               82       2.7k       100.0    82   2700
samneedsteeth               460       1.0k       469.0   460   1000
7l25lw                      173       1.4k       173.0   173   1400
7r70jw                      534       1.3k       540.0   534   1300
JamaicaWomensFootball        76        12k       108.0    76  12000
7zj4fo                     3.1k        12k      3175.0  3100  12000
8378h4                      419       1.1k       424.0   419   1100
83bxqs                      154       1.7k       167.0   154   1700
CPU times: user 5.73 s, sys: 21.8 s, total: 27.6 s
Wall time: 2min 28s


### Save progress

In [14]:
%%time
dfs_all_decent.to_csv(
    data_io.input_raw/'scrape_output'/'all_output_successful_and_cleaned.csv',
    encoding='utf-8',sep='|')

CPU times: user 47.8 s, sys: 10 s, total: 57.8 s
Wall time: 2min 13s


# 6. Drop duplicates by title,organizer, date created and location

In [4]:
dfs_all_decent = pd.read_csv(
    data_io.input_raw/'scrape_output'/'all_output_successful_and_cleaned.csv',
    index_col=[0],encoding='utf-8',sep='|')

/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (14,15,19,20,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Keep best duplicate by campaign_title, organizer, date created, and location

In [5]:
#### Keep best duplicate by campaign_title, organizer, date created, and location
dfs_all_decent = dfs_all_decent.assign(cleaned_location_city=dfs_all_decent.location_city.apply(lambda x: x.lower().strip()))
dfs_unique = utils.keep_best_duplicate(dfs_all_decent,subset=['title','poster','day','month','year',
                                                        'cleaned_location_city'],use_tqdm=use_tqdm)
#exclusion_df.loc['total','duplicate_title_organizer_date_loc'] = len(dfs_unique)
#exclusion_df.loc['deleted', 'duplicate_title_organizer_date_loc'] = len(dfs_all_decent) - len(dfs_unique)
print(dfs_all_decent.shape,dfs_unique.shape)

Processing duplicates:   0%|          | 0/564 [00:00<?, ?it/s]

(1621340, 32) (1574870, 32)


In [6]:
#exclusion_df.to_csv(data_io.input_cleaned/'gfm'/'exclusion_tracker_rd_1.csv')

Check to see if this is good criteria to identify duplicates, need to do this because sometimes the same campaign have different gfm_urls (when GFM move campaigns around or posters post twice) 

In [7]:
dfs_all_decent_gb = dfs_all_decent.groupby(['title','poster','day','month','year','cleaned_location_city'])
gb_size=dfs_all_decent_gb.size().sort_values(ascending=False)
any_nonu = gb_size>1
nonu_indexes =  any_nonu.index[any_nonu]

These duplicates should be the same campaign

In [8]:
gb_size[any_nonu] # groups that are not unique

title                                         poster                                     day   month     year    cleaned_location_city
Fund Name                                     COFIRSTNAME COLASTNAME                     2.0   May       2018.0  san diego, ca            12
                                                                                               June      2019.0  san diego, ca            12
                                                                                         21.0  January   2020.0  san diego, ca             9
Support the Creative Resistance!              Art and Resistance Through Education ARTE  21.0  December  2020.0  brooklyn, ny              8
Mizzou GW Beneficiary Fundraiser              Mizzou Greek Week                          20.0  March     2018.0  columbia, mo              7
                                                                                                                                          ..
Fighting Chronic Co

In [9]:
sample_dup=dfs_all_decent_gb.get_group(nonu_indexes[2]) # example of one group

In [10]:
sample_dup[['title','poster','day','month','year','location_city','story','goal']] 
# same campaign but diff campaign_id

,title,poster,day,month,year,location_city,story,goal
campaign_id,,,,,,,,
1vj19edn40,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vj19jffqo,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vj1ais05c,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vj5jf4ug0,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vj5jk6n2o,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vj5keheuo,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vjdt2yeio,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vjdt8075c,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none
1vjdu2ayxc,Fund Name,COFIRSTNAME COLASTNAME,21.0,January,2020.0,"San Diego, CA",This is a fund description,none


Optional: sort by created_date and campaign_id

In [11]:
dfs_unique.columns

Index(['url', 'last_donation_time', 'last_update_time', 'created_date',
       'location_city', 'location_country', 'location_postalcode',
       'location_stateprefix', 'poster', 'description', 'story', 'title',
       'goal', 'raised_amnt', 'goal_amnt', 'currency', 'tag', 'num_donors',
       'num_likes', 'num_shares', 'charity_details', 'error_message',
       'archive_timestamp', 'query_url', 'gfm_url', 'wayback_status',
       'i_filename', 'day', 'month', 'year', 'location_city_only',
       'cleaned_location_city'],
      dtype='object')

In [12]:
dfs_unique.sort_values(by=['year','month','day','campaign_id'],inplace=True)

In [13]:
dfs_unique.drop(columns=['cleaned_location_city','i_filename','wayback_status','error_message','archive_timestamp',
                        'query_url','description','gfm_url','location_postalcode','created_date','goal'],
                errors='ignore',
                inplace=True)
dfs_unique.to_csv(data_io.input_cleaned/'gfm'/'all_campaigns_successful_cleaned_noduplicate.csv',
                  encoding='utf-8',sep='|')

# 7. Narrow down to US cancer campaigns

### Narrow down to potentially US only campaigns

In [14]:
dfs_US_unique = dfs_unique.loc[dfs_unique.location_country.replace('none',np.nan).isna() | 
                               (dfs_unique.location_country=='US') ]

# additional filters 
valid_state_bool = dfs_US_unique.location_stateprefix.apply(utils.state_is_valid)
currency_is_USD = (dfs_US_unique.currency == 'USD') | (dfs_US_unique.currency == '$') | dfs_US_unique.currency.str.strip().replace('',np.nan).isna()

dfs_US_unique = dfs_US_unique.loc[valid_state_bool & currency_is_USD]
print(dfs_US_unique.shape,dfs_unique.shape)

(1265094, 21) (1574870, 21)


### Exclude any non medical tags and non-cancer campaigns

In [15]:
dfs_US_unique['is_cancer'] = dfs_US_unique.apply(lambda x: utils.find_cancer_story_title(x['story'], x['title']),axis=1)
dfs_US_unique['cancer_in_story'] = dfs_US_unique['story'].apply(utils.cancer_in_x)
dfs_US_unique['cancer_in_title'] = dfs_US_unique['title'].apply(utils.cancer_in_x)
print(dfs_US_unique['is_cancer'].value_counts())

dfs_US_unique['tag_is_medical'] = dfs_US_unique.tag.str.lower().apply(utils.assign_new_tag).notna()

# exclude campaigns set up for a charity, we only want individual campaigns
dfs_US_unique['is_for_charity'] = dfs_US_unique.charity_details.str.strip().replace(
    'none',np.nan).replace('',np.nan).notna()

dfs_US_cancer = dfs_US_unique.loc[dfs_US_unique.is_cancer & dfs_US_unique.tag_is_medical 
                                  & ~dfs_US_unique.is_for_charity]
dfs_US_cancer.drop(columns=['is_cancer','tag_is_medical','is_for_charity','charity_details'],
                   errors='ignore',inplace=True)

False    1059998
True      205096
Name: is_cancer, dtype: int64


/Users/Ryan/cancer-crowdfunding-explorer/gfm/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
dfs_US_unique.to_csv(data_io.input_cleaned/'gfm'/'US_campaigns_successful_cleaned_noduplicate.csv',
                  encoding='utf-8',sep='|')

In [17]:
dfs_US_cancer.to_csv(data_io.input_cleaned/'gfm'/'US_cancer_campaigns.csv',
                  encoding='utf-8',sep='|')

### Exclude by year 

In [18]:
# only want 2018 to 2021 campaigns
dfs_US_cancer_limit_year = dfs_US_cancer.loc[(dfs_US_cancer.year >=2018) & (dfs_US_cancer.year <=2021)]
print(dfs_US_cancer_limit_year.shape)

(86109, 22)


In [26]:
dfs_US_cancer_limit_year.groupby("year").size()

year
2018.0    21507
2019.0    22188
2020.0    35532
2021.0     6882
dtype: int64

In [19]:
dfs_US_cancer_limit_year.to_csv(data_io.input_cleaned/'gfm'/'US_cancer_campaigns_2018_2021.csv',
                  encoding='utf-8',sep='|')

### Generate unique locations to geocode later


In [20]:
unique_locs = dfs_US_cancer_limit_year[['location_city','location_stateprefix']].drop_duplicates()

unique_locs.to_csv(data_io.input_cleaned/'geolocations'/'unique_locations_from_US_cancer_campaigns_2018_2021.csv')

In [21]:
unique_locs = pd.read_csv(data_io.input_cleaned/'geolocations'/'unique_locations_from_US_cancer_campaigns_2018_2021.csv')

Get FIPs from old cancer campaign table

In [22]:
old_cancer = pd.read_excel(data_io.gfm/'cancer_w_locs_and_type_all_years_census_merged_v5.xlsx')

In [23]:
location_fip_map = old_cancer[['location','county_name','state_x','state_fips','county_fips',
                              'state_county_fips_str']].drop_duplicates()
location_fip_map.columns=location_fip_map.columns.map(
    location_fip_map.columns.to_series().replace('state_x','state').to_dict())

location_fip_map.state_county_fips_str = location_fip_map.state_county_fips_str.astype(str).str.zfill(5)

location_fip_map.to_csv(data_io.input_cleaned/'geolocations'/'unique_locations_w_fips.csv',
                                    encoding='utf-8',index=False)